# The following code requests data from themoviedb.org API and stores them in a PostgreSQL database

### 1. Import libraries

In [2]:
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import Date, Table, Column, Integer, String, ForeignKey, Text, ARRAY, Boolean, Float
import psycopg2
import urllib
import requests

### 2. Create database and table

In [ ]:
dbname = 'DBName'
username = 'postgres'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))

##### Create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)

##### Bind the connection to MetaData()
meta = sqlalchemy.MetaData(bind=engine, reflect=True)

##### Define table ####
film_table = Table('FilmTable', meta,
                   Column('Id', Integer, primary_key=True),
                   Column('imdb_id', String, default="None"),
                   Column('adult',Boolean, default=False),
                   Column('original_title', String, default="None"),
                   Column('genres', ARRAY(String), default="None"),
                   Column('genres_id', ARRAY(Integer), default=0),
                   Column('release_date', Date, default="None"),
                   Column('poster_path',String, default="None"),
                   Column('overview',String, default="None"),
                   Column('original_language', String, default="None"), 
                   Column('popularity', Float, default=0.),
                   Column('production_countries_iso_3166_1', ARRAY(String), default="None"),
                   Column('production_companies', ARRAY(String), default="None"),
                   Column('production_companies_id', ARRAY(Integer), default=0),
                   Column('backdrop_path',String, default="None"),
                   Column('belongs_to_collection', String, default="None"),
                   Column('belongs_to_collection_id', Integer, default=0),
                   Column('belongs_to_collection_backdrop_path', String, default="None"),
                   Column('belongs_to_collection_poster_path', String, default="None"),
                   Column('budget', Float, default=0.),
                   Column('homepage',String, default="None"),
                   Column('revenue', Float, default=0.),
                   Column('runtime', Float, default=0.),
                   Column('spoken_languages_iso_639_1', ARRAY(String), default="None"),
                   Column('status',String, default="None"),
                   Column('tagline',String, default="None"),
                   Column('title',String, default="None"),
                   Column('vote_average', Float, default=0.),
                   Column('vote_count', Integer, default=0),
                   Column('predictions', String, default="None"),
                   Column('fc2', String, default="None"),
                   Column('word2vec', String, default="None"), extend_existing=True)

##### Connect to database and create table
with engine.connect() as conn:
    film_table.create()


### 3. Request data from themoviedc.org API and store them in the database table

In [14]:
with engine.connect() as conn:

    for filein in range(0,300000):
        url = "https://api.themoviedb.org/3/movie/%s?language=en-US&api_key={key}" % filein
        payload = "{}"
        response = requests.request("GET", url, data=payload)
        ##### If url accessible, convert json file obtained to panda dataframe, and 
        ##### record relevant information for each movie
        
        if response.status_code == 200:
            responsepd=pd.Series(response.json())
            idfilm=responsepd['id']
            original_titlefilm=responsepd['original_title']
            imdb_idfilm=responsepd['imdb_id']
            adultfilm=False
            if(responsepd['adult']):
                adultfilm=responsepd['adult']
            release_datefilm="1900-01-01"    
            if(responsepd['release_date']):
                release_datefilm=responsepd['release_date']
            poster_pathfilm=responsepd['poster_path']
            overviewfilm=responsepd['overview']
            original_languagefilm=responsepd['original_language']
            popularityfilm=responsepd['popularity']
            backdrop_pathfilm=responsepd['backdrop_path']
            budgetfilm=responsepd['budget']
            homepagefilm=responsepd['homepage']
            revenuefilm=responsepd['revenue']
            runtimefilm=responsepd['runtime']
            statusfilm=responsepd['status']
            taglinefilm=responsepd['tagline']
            titlefilm=responsepd['title']
            vote_averagefilm=responsepd['vote_average']
            vote_countfilm=responsepd['vote_count']
            
            genrestab=list()
            genresidtab=list()           
            if (responsepd['genres']):
                for genreit in range(0,len(responsepd['genres'])):
                    genrestab.append(responsepd['genres'][genreit]['name'])
                    genresidtab.append(responsepd['genres'][genreit]['id'])
           
            production_countries_iso_3166_1tab=list()        
            if (responsepd['production_countries']):
                for production_countriesit in range(0,len(responsepd['production_countries'])):
                    production_countries_iso_3166_1tab.\
                    append(responsepd['production_countries'][production_countriesit]['iso_3166_1'])
            
            production_companiestab=list()
            production_companies_idtab=list()
            if (responsepd['production_companies']):
                for production_companiesit in range(0,len(responsepd['production_companies'])):
                    production_companiestab.\
                    append(responsepd['production_companies'][production_companiesit]['name'])
                    production_companies_idtab.\
                    append(responsepd['production_companies'][production_companiesit]['id'])
 
            belongs_to_collection_backdrop_pathfilm="None"
            belongs_to_collection_idfilm=0
            belongs_to_collectionfilm="None"
            belongs_to_collection_poster_pathfilm="None"

            if (responsepd['belongs_to_collection']):     
                belongs_to_collection_backdrop_pathfilm=responsepd['belongs_to_collection']['backdrop_path']
                belongs_to_collection_idfilm=responsepd['belongs_to_collection']['id']
                belongs_to_collectionfilm=responsepd['belongs_to_collection']['name']
                belongs_to_collection_poster_pathfilm=responsepd['belongs_to_collection']['poster_path']
               
            spoken_languages_iso_639_1tab=list()          
            if (responsepd['spoken_languages']):
                for spoken_languagesit in range(0,len(responsepd['spoken_languages'])):
                    spoken_languages_iso_639_1tab.append(responsepd['spoken_languages']\
                                                         [spoken_languagesit]['iso_639_1'])
                
            ##### Store Data into the table            
            insert_statement = film_table.insert()\
            .values(Id=idfilm,original_title=original_titlefilm, genres=genrestab, genres_id=genresidtab,\
                    imdb_id=imdb_idfilm, poster_path=poster_pathfilm,overview=overviewfilm,\
                    adult=adultfilm, release_date=release_datefilm,\
                    original_language=original_languagefilm,\
                    popularity=popularityfilm,\
                    production_countries_iso_3166_1=production_countries_iso_3166_1tab\
                    ,production_companies=production_companiestab, \
                    production_companies_id=production_companies_idtab, \
                    backdrop_path=backdrop_pathfilm, \
                    belongs_to_collection_backdrop_path=belongs_to_collection_backdrop_pathfilm,\
                    belongs_to_collection_id=belongs_to_collection_idfilm,\
                    belongs_to_collection=belongs_to_collectionfilm,\
                    belongs_to_collection_poster_path=belongs_to_collection_poster_pathfilm, \
                    budget=budgetfilm, homepage=homepagefilm, revenue=revenuefilm,\
                    runtime=runtimefilm,\
                    status=statusfilm, tagline=taglinefilm, title=titlefilm, \
                    vote_average=vote_averagefilm,\
                    vote_count=vote_countfilm, spoken_languages_iso_639_1=spoken_languages_iso_639_1tab)
    
            conn.execute(insert_statement)     
            

### 4. Verify the table was created and filled properly

In [18]:
########### Show PostgresSQL table using pandas ##########

with engine.connect() as conn:
    select_statement = film_table.select()
    pdfilm_table_from_sql = pd.read_sql_query(select_statement,conn)

pdfilm_table_from_sql.head()

,Id,imdb_id,adult,original_title,genres,genres_id,release_date,poster_path,overview,original_language,...,runtime,spoken_languages_iso_639_1,status,tagline,title,vote_average,vote_count,predictions,fc2,word2vec
0,2,tt0094675,False,Ariel,"[Drama, Crime]","[18, 80]",1988-10-21,/gZCJZOn4l0Zj5hAxsMbxoS6CL0u.jpg,Taisto Kasurinen is a Finnish coal miner whose...,fi,...,69.0,"[fi, de]",Released,,Ariel,7.1,42,None,None,None
1,3,tt0092149,False,Varjoja paratiisissa,"[Drama, Comedy]","[18, 35]",1986-10-16,/7ad4iku8cYBuB08g9yAU7tHJik5.jpg,"An episode in the life of Nikander, a garbage ...",fi,...,76.0,"[en, fi, sv]",Released,,Shadows in Paradise,7.0,34,None,None,None
2,5,tt0113101,False,Four Rooms,"[Crime, Comedy]","[80, 35]",1995-12-09,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,It's Ted the Bellhop's first night on the job....,en,...,98.0,[en],Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,535,None,None,None
3,6,tt0107286,False,Judgment Night,"[Action, Thriller, Crime]","[28, 53, 80]",1993-10-15,/lNXmgUrP6h1nD53gkFh4WDzT6RZ.jpg,"While racing to a boxing match, Frank, Mike, J...",en,...,110.0,[en],Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.4,77,None,None,None
4,8,tt0825671,False,Life in Loops (A Megacities RMX),[Documentary],[99],2006-01-01,/8YyIjOAxwzD3fZMdmJrfiApod4l.jpg,Timo Novotny labels his new project an experim...,en,...,80.0,"[en, hi, ja, ru, es]",Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,None,None,None
